In [184]:
import numpy as np
import copy
import math
import scipy.linalg as la
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint

First define all variables:

In [185]:
# Setting size of the network
j = 2 # number of junctions
z = 3 # number of links per junction
i = 2 # number of inflowing links per junction, has to be smaller than z
o = 1 # number of outflow links

# Inflowing links
I = np.ones((j,z)) # placeholder for inflowing links matrix
I[:,i:z] = 0 # set to zero for outflowing links per junction (assumes inflowing links have no outflowing traffic)

# Outflowing links
O = np.zeros((j,z)) # placeholder for inflowing links matrix
O[:,z-o:o] = 1 # set to zero for outflowing links per junction (assumes inflowing links have no outflowing traffic)

# Saturation flows
S = np.ones((j,z)) # Saturation flow for each incoming link

# Adjacency matrix
N = np.zeros((j*z,j*z)) # placeholder for junction adjacency
connections = [(z-1,z)] # only connection in this setup is from junction 1 link 3 to junction 2 link 1 (excluding vice versa)
for each in connections:
    N[each] = 1 # apply connections in adjacency matrix

# Setting time frame of the network
C = np.full((j),2) # cycle time
L = np.full((j),0.5) # lost time
T = 10 # control cycle

# Setting stage numbers 
i_s = 2 # number of stages per inflowing link
i_g = 1 # number of stages where inflowing link has right of way
F = np.zeros((j,z,i_s)) # matrix of stages per link
F[:,:,i_g] = 1 # setting green stages per link

# Setting green times and turning rates
G = np.ones((j,i)) # green time matrix
Tu = np.ones((j,i,o)) # turning rate matrix
for junction in range(j):
    G[junction,:] = G[junction,:] * np.random.random((i)) # assign random green times at each junction's incoming links
    Tu[junction,:] = Tu[junction,:] * np.random.random((i,o)) # assign a random turning rate to each junction's incoming/outflowing link combination
    Tu[junction,:] = 1 / np.sum(Tu[junction,:]) * Tu[junction,:] # making sure total turning rates are not larger than 1
Tu0 = np.ones(j) # turners who leave the system
Tu0 = Tu0*0.5 # balance with inflow
    
# demand and exit flows per junction
d = np.ones(j)*1.5 # steady-state demand flow: balances with turners who leave system

Now define the system matrices

In [186]:
# System matrices
A1 = np.zeros(j) # initialise one A1 for each junction
A2 = np.zeros(j) # initialise one A1 for each junction
A3 = np.zeros(j) # initialise one A1 for each junction
for junction in range(j):
    A1[junction] = (1-Tu0[junction]) * (Tu[junction,0,0] * S[junction,0] / C[junction])
    A2[junction] = (1-Tu0[junction]) * (Tu[junction,1,0] * S[junction,1] / C[junction])
    A3[junction] = (1-Tu0[junction]) * (S[junction,2] / C[junction])
    
X = np.full((j*o),10)  # initialise number of vehicles per outflow link

Introduce the dynamics

In [187]:
# Dynamics
X = X + T * (A1 * G[:,0] + A2 * G[:,1]  + A3 * np.array([G[0,0],1])) # when system is scaled up need to read third multiplier from adjacency matrix

Questions pending: How do we calculate the steady state green rates?

Comments

- G matrix should have 5 values? Exit green light?
- Should we define times in sec?

Solving Riccati equations

In [1]:
# Riccati matrices --> Have to be obtained from Seb's code
A = np.identity(2)

B = np.array([[A1[0], A2[0], A3[0]  ,0     ,  0    ],
              [0    , 0    , A1[1]  ,A2[1] ,  A3[1]]]) #check!!!!

Q = np.identity(2)/100
R = np.identity(5)*1e-4

aux = la.solve_discrete_are(A,B,Q,R) # Solving Riccati equations
L_ric = la.inv(R + B.T.dot(aux).dot(B)) @ B.T.dot(aux).bdot(A) # Finding L

delta_g = L_ric @ X.T 

NameError: name 'A1' is not defined

In [2]:
gn = np.ones((5))*10
g = (gn - delta_g)

NameError: name 'delta_g' is not defined

Solving taking into account signal constraints

In [3]:
# Function to be minimized f
def greens(G):
    return (np.sum((g-G)**2))

# Gradient of f
def greens_grad(G):
    return (-2*(g-G))

# Hessian of f
def hess(G):
    return (np.identity(5)*2)


# Defining feasible times for green lights
bounds = ((-1,1),(-1,1),(-1,1),(-1,1),(-1,1))

# Constraint to make sure that all signals in a link stay within the cycle defined time.
linear_constraint = ({'type': 'eq', 'fun': lambda x:  x[0] + x[1] + L[0] - C[0]}, # Link 1
                     {'type': 'eq', 'fun': lambda x: x[2] + x[3] + L[1] - C[1]}, # Link 2
                     {'type': 'eq', 'fun': lambda x: x[4] + 2}) # Exit link

x0 = np.array([1, 1, 1, 1, 1]) #initialization

minimize(greens, # function to be minimized
         x0=x0, # init point
         method='SLSQP', # Sequential Least SQuares Programming method for minimizing
         jac = greens_grad, # Gradient of the function
         hess = hess, # Hessian
         constraints = linear_constraint, # Linear constraints
         bounds = bounds) # Bounds

NameError: name 'minimize' is not defined